In [71]:
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2 as tv2
import pandas as pd
from PIL import Image
from tqdm import tqdm
from typing import Callable
from pathlib import Path

In [72]:
ROOT = "/home/oedada/Projects/experiments/NTO/Final/crack"
SIZE = (128, 128)
BATCH_SIZE = 30

In [ ]:
df  = pd.read_csv(ROOT + "data/annotations.csv")
df.head()

,image,road_health
0,images/c4d20454-29f5-4d28-bebf-9c5c9942478d.jpg,0.60
1,images/d9c36fc1-2977-4c6e-b060-51eb04715aeb.jpg,0.85
2,images/27d03ad4-070e-4fbd-a902-e4501e24fb41.jpg,0.50
3,images/366c957f-854d-458d-87c4-668f81ea2fad.jpg,1.00
4,images/02d50cbf-5940-42fe-b0fd-8e758d97ac4e.jpg,0.85


In [74]:
class CrackDataset:
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = Image.open(ROOT + "/" + self.df.iloc[idx]["image"]).convert("RGB")
        if self.transform:
            img = self.transform(img)

        return (img, float(self.df.iloc[idx]["road_health"]))


In [75]:
train_trans = tv2.Compose([
    tv2.Resize(SIZE),
    tv2.RandomVerticalFlip(),
    tv2.RandomHorizontalFlip(),
    # tv2.ColorJitter(brightness=0.2, contrast=0.2),
    tv2.ToImage(),
    tv2.ToDtype(torch.float32, scale=True)
])
val_trans = tv2.Compose([
    tv2.Resize(SIZE),
    tv2.ToImage(),
    tv2.ToDtype(torch.float32, scale=True)
])


In [76]:
train_df = df.iloc[:int(0.9 * len(df))]
val_df   = df.iloc[int(0.9 * len(df)):]

train_dataset = CrackDataset(train_df, transform=train_trans)
val_dataset = CrackDataset(val_df, transform=val_trans)

In [77]:
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
)

In [82]:
import torch.nn as nn
#padding = (kernel_size - 1) // 2
class RoadRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            # nn.Dropout2d(0.1)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            # nn.Dropout2d(0.1)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            # nn.Dropout2d(0.1)
        )
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, 64),
            # nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.fc(x)
        return x


In [ ]:
class CompiledModel():
    '''
    Класс для облегченной тренировки и валидации модели типа tf.compile()
    '''

    def __init__(self, model, optimizer, loss_function: Callable, model_name: str = None):
        self.original_model = model
        self.model = model
        self.optim = optimizer
        self.lossf = loss_function
        self.num_epoch = 1
        if model_name is not None:
            self.name = str(model_name)
        else:
            self.name = str("model")

    def clear_model(self):
        self.model = self.original_model
        self.num_epoch = 1

    def train(self, epochs: int, train_data_loader, verbose: bool, val_data_loader = None, save_dir: str | Path = None):
        for epoch in range(self.num_epoch, epochs+self.num_epoch):
            self.model.train()
            if verbose:
                train_bar = tqdm(train_data_loader, desc=f"Training Epoch: {epoch}/{epochs}", leave=False, unit="batch")
            run_loss = 0
            for x, y in train_bar:
                predict = self.model(x)
                loss = self.lossf(predict, y.unsqueeze(dim=1).float())
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                loss_item = loss.item()
                run_loss += loss_item*train_dl.batch_size
                if verbose:
                    train_bar.set_postfix(loss=f"{loss_item:.4f}")
            if val_data_loader is not None:
                val_accurancy = self.evaluate(val_data_loader, verbose=verbose)
            print("\r" + f"Epoch: {epoch}/{epochs}, train_loss = {run_loss/len(train_dl.dataset):.4f}, val_MAE = {val_accurancy:.4f}")
            if save_dir is not None:
                torch.save(model, Path(save_dir) / (self.name + f"_{self.num_epoch}.pth"))
            self.num_epoch += 1

    def evaluate(self, val_data_loader, verbose: bool = False):
        self.model.eval()
        if verbose:
            val_bar = tqdm(val_data_loader, desc="Validation", leave=False, unit="batch")
        n_samples = 0
        total_error = 0
        with torch.no_grad():
            for i, (x, y) in enumerate(val_bar):
                p = model(x)
                y = y.unsqueeze(dim=1)
                batch_error = torch.abs(p - y).sum().item()
                total_error += batch_error
                n_samples += x.size(0)
                if verbose:
                    val_bar.set_postfix(accurancy = f"{total_error/n_samples:.4f}")
        return total_error/n_samples

In [85]:
model = RoadRegressor()

loss_func = nn.MSELoss()

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=2e-4
)

In [89]:
mod = CompiledModel(model, optimizer, loss_function=loss_func, model_name="crack")
mod.train(30, train_dl, True, val_dl, save_dir="models")

Epoch: 1/30, train_loss = 0.0016, val_MAE = 0.0363


Epoch: 2/30, train_loss = 0.0018, val_MAE = 0.0326


Epoch: 3/30, train_loss = 0.0015, val_MAE = 0.0297


Epoch: 4/30, train_loss = 0.0014, val_MAE = 0.0319


Epoch: 5/30, train_loss = 0.0014, val_MAE = 0.0315


...

Epoch: 26/30, train_loss = 0.0005, val_MAE = 0.0171


Epoch: 27/30, train_loss = 0.0006, val_MAE = 0.0179


Epoch: 28/30, train_loss = 0.0006, val_MAE = 0.0177


Epoch: 29/30, train_loss = 0.0005, val_MAE = 0.0177


Epoch: 30/30, train_loss = 0.0004, val_MAE = 0.0181


In [90]:
model_l = torch.load("/home/oedada/Projects/experiments/NTO/Final/crack/models/crack_24.pth", weights_only=False)

In [93]:
model_l.eval()
val_bar = tqdm(val_dl, desc="Validation", unit="batch")
n_samples = 0
total_error = 0
with torch.no_grad():
    for i, (x, y) in enumerate(val_bar):
        p = model_l(x)
        y = y.unsqueeze(dim=1)
        batch_error = (torch.abs(p - y) < 0.05).sum().item()
        total_error += batch_error
        n_samples += x.size(0)
        val_bar.set_postfix(accurancy = f"{total_error/n_samples:.4f}")

Validation: 100%|██████████| 3/3 [00:00<00:00,  4.82batch/s, accurancy=0.9222]


In [95]:
model_l.eval()
val_bar = tqdm(train_dl, desc="Validation", unit="batch")
n_samples = 0
total_error = 0
with torch.no_grad():
    for i, (x, y) in enumerate(val_bar):
        p = 0.5
        y = y.unsqueeze(dim=1)
        batch_error = (torch.abs(p - y) < 0.05).sum().item()
        total_error += batch_error
        n_samples += x.size(0)
        val_bar.set_postfix(accurancy = f"{total_error/n_samples:.4f}")

Validation: 100%|██████████| 27/27 [00:02<00:00, 11.70batch/s, accurancy=0.1383]
